# Lab 2: DAGs và Tasks - Tạo Workflows với Task SDK

## 🎯 Objectives
- Học cách tạo DAGs với Task SDK (@dag, @task decorators)
- Hiểu cách định nghĩa tasks và dependencies
- Xử lý errors và retries
- Tạo ETL pipeline đơn giản

## 📋 Prerequisites
- Hoàn thành Lab 1: Airflow Basics
- Airflow cluster đang chạy
- Basic Python knowledge

## 🏗️ Task SDK Overview
Airflow 3.x giới thiệu Task SDK với decorators:
- `@dag`: Định nghĩa DAG
- `@task`: Định nghĩa Python task
- `@task.bash`: Định nghĩa bash task
- `@task.docker`: Định nghĩa Docker task


## 1. Import Libraries


In [ ]:
# Import Airflow Task SDK
from airflow.sdk import DAG, task
from airflow.sdk.task import Task
import pendulum
from datetime import datetime, timedelta

print("✅ Airflow Task SDK imported successfully!")
print(f"📦 Airflow version: Check Airflow UI or CLI")


## 2. Tạo DAG đơn giản với @dag decorator


In [ ]:
# Tạo DAG đơn giản với @dag decorator
@dag(
    dag_id="simple_dag_example",
    schedule=None,  # Manual trigger
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["tutorial", "example"],
)
def simple_dag():
    """
    ### Simple DAG Example
    DAG đơn giản với một task duy nhất.
    """
    
    @task
    def hello_task():
        """Print hello message"""
        print("Hello from Airflow Task SDK!")
        return "Task completed"
    
    hello_task()

# Tạo DAG instance
dag_instance = simple_dag()

print("✅ DAG created successfully!")
print(f"DAG ID: {dag_instance.dag_id}")
print(f"Tasks: {[task.task_id for task in dag_instance.tasks]}")


## 3. Tạo ETL Pipeline với Task Dependencies

Chúng ta sẽ tạo một ETL pipeline đơn giản với 3 tasks:
1. Extract: Lấy dữ liệu
2. Transform: Biến đổi dữ liệu
3. Load: Lưu dữ liệu


In [ ]:
# ETL Pipeline Example
@dag(
    dag_id="etl_pipeline_example",
    schedule="@daily",
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["etl", "pipeline"],
)
def etl_pipeline():
    """
    ### ETL Pipeline
    Extract, Transform, Load pipeline example.
    """
    
    @task
    def extract():
        """Extract data from source"""
        print("Extracting data...")
        # Simulate data extraction
        data = {
            "users": [
                {"id": 1, "name": "Alice", "age": 30},
                {"id": 2, "name": "Bob", "age": 25},
                {"id": 3, "name": "Charlie", "age": 35},
            ],
            "timestamp": datetime.now().isoformat(),
        }
        print(f"Extracted {len(data['users'])} records")
        return data
    
    @task
    def transform(data: dict):
        """Transform extracted data"""
        print("Transforming data...")
        users = data["users"]
        
        # Calculate statistics
        total_age = sum(user["age"] for user in users)
        avg_age = total_age / len(users)
        
        transformed = {
            "total_users": len(users),
            "average_age": avg_age,
            "timestamp": data["timestamp"],
        }
        print(f"Transformed: {transformed}")
        return transformed
    
    @task
    def load(data: dict):
        """Load transformed data to destination"""
        print("Loading data...")
        print(f"Loading {data['total_users']} users with avg age {data['average_age']:.2f}")
        return f"Loaded {data['total_users']} records successfully"
    
    # Define dependencies
    extracted_data = extract()
    transformed_data = transform(extracted_data)
    load(transformed_data)

# Create DAG
etl_dag = etl_pipeline()

print("✅ ETL Pipeline DAG created!")
print(f"Tasks: {[task.task_id for task in etl_dag.tasks]}")


## 4. Error Handling và Retries

Airflow hỗ trợ retry logic để xử lý failures tự động.


In [ ]:
# DAG với retry logic
@dag(
    dag_id="retry_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    default_args={
        "retries": 3,  # Retry 3 times
        "retry_delay": timedelta(minutes=1),  # Wait 1 minute between retries
    },
    tags=["retry", "error-handling"],
)
def retry_example():
    """
    ### Retry Example
    DAG với retry logic để xử lý failures.
    """
    
    @task(retries=2, retry_delay=timedelta(seconds=30))
    def unreliable_task():
        """Task có thể fail"""
        import random
        
        # Simulate random failure (50% chance)
        if random.random() < 0.5:
            print("Task failed! Will retry...")
            raise Exception("Random failure occurred")
        else:
            print("Task succeeded!")
            return "Success"
    
    @task
    def always_succeed():
        """Task luôn thành công"""
        print("This task always succeeds")
        return "Done"
    
    unreliable_task() >> always_succeed()

# Create DAG
retry_dag = retry_example()

print("✅ Retry Example DAG created!")
print("💡 This DAG demonstrates retry logic for error handling")


## 5. Tóm tắt và Next Steps

### ✅ Những gì đã học:
1. Tạo DAGs với @dag decorator
2. Tạo tasks với @task decorator
3. Định nghĩa task dependencies
4. Xử lý errors với retries

### 📚 Next Lab:
- **Lab 3**: Operators và Hooks
- Sử dụng BashOperator, PythonOperator
- SQLExecuteQueryOperator
- Custom operators và hooks

### 🔗 Useful Links:
- [Task SDK Documentation](https://airflow.apache.org/docs/apache-airflow/3.1.1/task-sdk/index.html)
- [DAG Best Practices](https://airflow.apache.org/docs/apache-airflow/3.1.1/best-practices/index.html)
